In [1]:
import numpy as np
import matplotlib as plt
import pandas as pd

In [2]:
path = "/home/taha/Documents/ML/iris.txt"
df = pd.read_csv(path)

# Column labels.
headers = ["sepal length", "sepal width", "petal length", "petal width" , "label"]

df.columns = headers

# Converting labels to numeric.
label_map = {"Iris-setosa":0, "Iris-versicolor":1, "Iris-virginica":2}
df['label'] = df['label'].map(label_map)

train = df.sample(frac=0.8,random_state=200)
test = df.drop(train.index)

training_dataset = train.values
testing_dataset = test.values

In [3]:
classes = 3
X = training_dataset[:, :-1].T
Y_orig = (training_dataset[:, -1].reshape(1,X.shape[1])).astype(int)
X_test = testing_dataset[:, :-1].T
Y_test_orig = (testing_dataset[:, -1].reshape(1,X_test.shape[1])).astype(int)

m = X.shape[1]

#Modifying Y for multi class Classification
Y = np.zeros((m, classes))
Y[np.arange(m), Y_orig] = 1
Y = Y.T

Y_test = np.zeros((X_test.shape[1], classes))
Y_test[np.arange(X_test.shape[1]), Y_test_orig] = 1
Y_test = Y_test.T


print ('The shape of X is: ' + str(X.shape))
print ('The shape of Y is: ' + str(Y.shape))
print ('I have m = %d training examples!' % (m))


The shape of X is: (4, 119)
The shape of Y is: (3, 119)
I have m = 119 training examples!


In [4]:
def layer_sizes(X, Y):
    n_x = X.shape[0] # size of input layer
    n_y = Y.shape[0] # size of output layer
            
    return (n_x, n_y)

In [5]:
def initialize_parameters(n_x,n_y):
    
    W = np.random.randn(n_y,n_x) * 0.01
    b = np.zeros((n_y,1)) 
    
    parameters = {"W": W,
                  "b": b}
    
    return parameters

In [6]:
def sigmoid(X):
    
    return 1.0 / (1.0 + np.exp(-1.0 * X))

In [7]:
def softmax(X):
    exps = np.exp(X)
    return exps / exps.sum(axis=0, keepdims=True)

In [8]:
def forward_propagation(X, parameters):
    
    W = parameters['W']
    b = parameters['b']
    
    Z = np.dot(W,X) + b
    A = softmax(Z)
    
    cache = {"Z": Z,
             "A": A}
    
    return A, cache

In [9]:
def compute_cost(A, Y, parameters):
    
    m = Y.shape[1]
    
    logprobs = np.multiply(np.log(A),-Y)
    #column wise
    cost = np.sum(logprobs,axis=0,keepdims=True)
    #over all training examples
    cost = np.sum(cost)/m
    cost = np.squeeze(cost)
    return cost

In [10]:
def backward_propagation(parameters, cache, X, Y):
    
    m = X.shape[1]
    
    W = parameters['W']
    
    A = cache['A']
    
    dZ = A - Y
    dW = (1/m) * np.dot(dZ,X.T)
    db = (1/m) * np.sum(dZ,axis=1,keepdims=True)
    
    grads = {"dW": dW,
             "db": db}
    
    return grads

In [11]:
def update_parameters(parameters, grads, learning_rate = 0.5):
    
    W = parameters['W']
    b = parameters['b']
    
    dW = grads['dW']
    db = grads['db']
   
    # Update rule for each parameter
    W = W - learning_rate * dW
    b = b - learning_rate * db
    
    parameters = {"W": W,
                  "b": b}
    
    return parameters

In [12]:
def nn_model(X, Y, num_iterations = 10000, print_cost=False):
    
    n_x = layer_sizes(X, Y)[0]
    n_y = layer_sizes(X, Y)[1]
    
    parameters = initialize_parameters(n_x,n_y)
    W = parameters['W']
    b = parameters['b']
    
    for i in range(0, num_iterations):
        
        # Forward propagation
        A, cache = forward_propagation(X, parameters)
   
        # Cost function
        cost = compute_cost(A, Y, parameters)
 
        # Backpropagation
        grads = backward_propagation(parameters, cache, X, Y)
 
        # Gradient descent parameter update
        parameters = update_parameters(parameters, grads)
        
        # Print the cost every 1000 iterations
        if print_cost and i % 1000 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))

    return parameters

In [13]:
def predict(parameters, X):
    
    A, cache = forward_propagation(X, parameters)
    predictions = (A == A.max(axis=0)[None,:]).astype(int)
    
    return predictions

In [14]:
parameters = nn_model(X, Y,num_iterations = 10000, print_cost=True)

Cost after iteration 0: 1.102285
Cost after iteration 1000: 0.074236
Cost after iteration 2000: 0.068664
Cost after iteration 3000: 0.065081
Cost after iteration 4000: 0.062460
Cost after iteration 5000: 0.060447
Cost after iteration 6000: 0.058845
Cost after iteration 7000: 0.057533
Cost after iteration 8000: 0.056432
Cost after iteration 9000: 0.055492


In [15]:
# Print accuracy
predictions = predict(parameters, X_test)
correct_classified = np.sum(np.multiply(Y_test,predictions),axis=0)

print ('Accuracy: %d' % float( np.sum(correct_classified)/float(Y_test.shape[1])*100) + '%')

Accuracy: 100%
